In [ ]:
# pepsi.evaluation.ipynb
# by: Tim Lucas Halt (MatrNr. 6682645)

# Vorbereitung
Zunächst werden die notwendigen Bibliotheken installiert und importiert. Möglicherweise ist ein Neustart des Kernels erforderlich.

In [ ]:
%pip install scipy
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install tensorflow
%pip install sweetviz

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sweetviz as sw
import seaborn as sns
sns.set()

In [31]:
# Let's define the URL for the schiller dataset:
path = "https://raw.githubusercontent.com/schutera/tiny_schiller/main/tiny_schiller.txt"

# Let's download the dataset and save it as 'shakespeare.txt':
filepath = tf.keras.utils.get_file("schiller.txt", path)

# open and read the downloaded text file
with open(filepath) as f:
    schiller_text = f.read()

print(schiller_text[:100])

Friedrich Schiller
Briefe über Don Carlos.
Erster Brief.
Sie sagen mir, lieber Freund, daß Ihnen die


In [32]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(schiller_text)

max_id=len(tokenizer.word_index)
dataset_size=tokenizer.document_count
[encoded]=np.array(tokenizer.texts_to_sequences([schiller_text]))-1

In [49]:
train_size=dataset_size*80//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps=100
window_length=n_steps+1
dataset=dataset.repeat().window(window_length,shift=1,drop_remainder=True)

dataset=dataset.flat_map(lambda window: window.batch(window_length))

batch_size=1024
dataset=dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))
dataset=dataset.map(lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch))
dataset=dataset.prefetch(1)

In [50]:
model=keras.models.Sequential([
    keras.layers.GRU(256, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')
history=model.fit(dataset,
                  steps_per_epoch=train_size/batch_size,
                  epochs=10)

Epoch 1/10
1594/1594 [==============================] - 382s 236ms/step - loss: 1.9943
Epoch 2/10
1594/1594 [==============================] - 373s 234ms/step - loss: 1.7113
Epoch 3/10
 535/1594 [=========>....................] - ETA: 4:06 - loss: 1.7051

In [45]:
def preprocess(texts):
    X=np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X,max_id)

def next_char(text,temperature=1):
    X_new=preprocess([text])
    y_proba=model.predict(X_new)[0,-1:,:]
    rescaled_logits=tf.math.log(y_proba)/temperature
    char_id=tf.random.categorical(rescaled_logits,num_samples=1)+1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text,n_chars=50,temperature=0.1):
    for _ in range(n_chars):
        text+=next_char(text,temperature)
    return text

In [47]:
print("Some predicted texts for word 'Deep Learning' are as follows:\n ")
for i in range(3):
  print(complete_text('Deep Learning'))
  print()

Some predicted texts for word 'MNachttischlampe' are as follows:
 
1/1 [==============================] - 0s 18ms/step
Nachttischlampern, die sechs fürsten rasch zu denkst, oder fin ge

1/1 [==============================] - 0s 18ms/step
Nachttischlampenbist werde ich mein elend, wir drohen, die wink i

1/1 [==============================] - 0s 28ms/step
Nachttischlampe, davonten weeder um sie wielig für eine gedechtig

